In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csc_matrix

In [2]:
data_dir = '../data/processed/'

In [3]:
rating_df = pd.read_csv(data_dir + 'train_rating.csv')
rating_df.shape

(12800168, 5)

In [4]:
mean_ratings = rating_df.groupby('movieId').rating.mean().reset_index()
mean_ratings.head()

,movieId,rating
0,1,3.920535
1,2,3.209256
2,3,3.146716
3,4,2.856856
4,5,3.070523


In [5]:
movies_df = pd.read_csv(data_dir + 'movie.csv')
# movies_df = movies_df[movies_df.movieId.isin(mean_ratings.movieId)]
movies_df.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,imax,musical,mystery,romance,scifi,thriller,unknown,war,western,year
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1995.0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1995.0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0


Filtrar filmes que estão nos dados de treino

In [90]:
movies_train = movies_df[movies_df.movieId.isin(mean_ratings.movieId)]

In [91]:
movies_train[['movieId','title']].sample(10)

,movieId,title
1942,2026,Disturbing Behavior (1998)
2279,2364,Godzilla 1985: The Legend Is Reborn (Gojira) (...
7273,7385,Twentynine Palms (2003)
6512,6622,Carnages (a.k.a. Carnage) (2002)
18780,93492,Like Water (2011)
20264,99675,Eat Sleep Die (Äta sova dö) (2012)
18125,90746,"Adventures of Tintin, The (2011)"
10546,39824,Mahogany (1975)
22260,107059,Dead Silence (1997)
20060,98850,Dr. Wai in the Scriptures with No Words (Mao x...


In [92]:
cross = movies_train.merge(mean_ratings, how='inner', on='movieId')
cross.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,musical,mystery,romance,scifi,thriller,unknown,war,western,year,rating
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1995.0,3.920535
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1995.0,3.209256
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1995.0,3.146716
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,1995.0,2.856856
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1995.0,3.070523


In [93]:
cross.shape

(24895, 24)

In [94]:
null_year = cross[cross.year.isna()].index
null_year.shape

(15,)

In [95]:
cross.drop(index=null_year, inplace=True)

In [96]:
cross.drop(columns='title', inplace=True)
# cross.drop(columns='movieId', inplace=True)

In [97]:
from sklearn.preprocessing import MinMaxScaler

In [98]:
cross.head()

,movieId,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,...,musical,mystery,romance,scifi,thriller,unknown,war,western,year,rating
0,1,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1995.0,3.920535
1,2,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1995.0,3.209256
2,3,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1995.0,3.146716
3,4,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1995.0,2.856856
4,5,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1995.0,3.070523


In [99]:
scaler = MinMaxScaler()
# removing movieId
scaler.fit(cross.iloc[:, 1:])
features = scaler.transform(cross.iloc[:, 1:])

In [100]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(
    # Quantidade de recomendações
    n_neighbors=6,
    # Algoritmo
    algorithm='brute',
    # metrica de distancia
    metric='minkowski'
)

model.fit(features)

NearestNeighbors(algorithm='brute', n_neighbors=6)

In [101]:
# toy story movie
sample = cross[cross.movieId == 1].iloc[:, 1:]
sample = scaler.transform(sample)

In [102]:
distances, indexes = model.kneighbors(sample)
indexes = indexes[0]
indexes

array([    0,  3027,  4789,  3921, 24875, 10063])

In [104]:
recs = cross.loc[indexes].merge(movies_train[['movieId','title']], on='movieId', how='inner')

In [105]:
recs[['title']]

,title
0,Toy Story (1995)
1,Toy Story 2 (1999)
2,"Monsters, Inc. (2001)"
3,"Emperor's New Groove, The (2000)"
4,7 Dwarves: The Forest Is Not Enough (2006)
5,DuckTales: The Movie - Treasure of the Lost La...
